In [2]:
import os
import re
import glob
import pickle
import numpy as np
import pandas as pd
import pandas.io.common

In [3]:
#### File Path and Variables
loc_path = r'/Users/raj2.gaurav/Desktop/' 
path = loc_path + r'/Causal Model/01. Raw Data/03. LSR Data/'
pickle_path = loc_path + 'Causal Model/02. Processed Data/03. LSR Data/'
### Input for the Circle 
Circle= 'bihar'
monthname='Nov'
year = '2022'

In [4]:
## Pre Processing
def remove_whitespaces(df):
    df_obj = df.select_dtypes(['object'])
    df[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())
    df.columns = df.columns.str.strip()
    return df

def drop_duplicate_records(df):
    df.drop_duplicates()
    return df

#### Collating Files 


allFiles = glob.glob(path + "/*.parquet")
lst=[]
df_master=pd.DataFrame()
for i in allFiles:
    df=pd.read_parquet(i)
    #print(df.shape)
    df_master=df_master.append(df)
    #print(df_master.shape)

/var/folders/3p/7q9_hy6x4q58xrz02_n9tvv0fryx5_/T/ipykernel_22495/1357199947.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_master=df_master.append(df)
/var/folders/3p/7q9_hy6x4q58xrz02_n9tvv0fryx5_/T/ipykernel_22495/1357199947.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_master=df_master.append(df)
/var/folders/3p/7q9_hy6x4q58xrz02_n9tvv0fryx5_/T/ipykernel_22495/1357199947.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_master=df_master.append(df)
/var/folders/3p/7q9_hy6x4q58xrz02_n9tvv0fryx5_/T/ipykernel_22495/1357199947.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_master=df_master.append(df)
/var

In [5]:
df_master.head()

,circle,city,partition_date,component,hour,sum_total_volume_MB_download,sum_total_volume_MB_upload,unique_customers,total_duration_sec,total_sessions,...,rsrq_nom,rsrq_dnom,mean_cqi_nom,mean_cqi_dnom,pusch_sinr_nom,pusch_sinr_dnom,ta_distance_nom,ta_distance_dnom,intersite_distance_nom,intersite_distance_dnom
0,bihar,hajipur,2022-11-04,98.0,17,44547835768.00,4727227696.00,4672,840864,49706,...,-562649.00,39294,392691.0,49276,146593.70,12682,56784132.00,49706,83771883.00,49706
1,bihar,jamui,2022-11-04,56.0,17,45228882630.00,4137939740.00,5133,1598569,43598,...,-398315.50,28126,376384.0,43409,326344.50,22993,13896817.00,43598,12780028.00,43598
2,bihar,daltonganj,2022-11-04,9.0,17,42583194734.00,3876797554.00,4902,1257292,43465,...,-461172.50,34509,389276.0,43192,297332.90,19718,10627596.00,43465,11452453.00,43465
3,bihar,sitamarhi,2022-11-04,27.0,17,75229462619.00,6707864085.00,8134,2627563,78809,...,-830623.50,55742,622129.0,78306,493853.90,42966,42858573.00,78809,33896908.00,78809
4,bihar,gaya,2022-11-04,112.0,17,110061710390.00,10600855782.00,8041,3021133,89193,...,-799238.00,60384,806191.0,88822,635052.00,45805,20777164.00,89193,26351571.00,89193


## Pre Process

In [6]:
print(df_master.shape)
print(df_master.columns)

(964712, 22)
Index(['circle', 'city', 'partition_date', 'component', 'hour',
       'sum_total_volume_MB_download', 'sum_total_volume_MB_upload',
       'unique_customers', 'total_duration_sec', 'total_sessions', 'rsrp_nom',
       'rsrp_dnom', 'rsrq_nom', 'rsrq_dnom', 'mean_cqi_nom', 'mean_cqi_dnom',
       'pusch_sinr_nom', 'pusch_sinr_dnom', 'ta_distance_nom',
       'ta_distance_dnom', 'intersite_distance_nom',
       'intersite_distance_dnom'],
      dtype='object')


In [7]:
print('Removing right Spaces from the columns')
df_master.columns = df_master.columns.str.rstrip()
df_master.columns = df_master.columns.str.lstrip()
df_master.columns = df_master.columns.str.capitalize()

Removing right Spaces from the columns


In [8]:
print('Size of the Raw Data ',df_master.shape)
print('-------------------------------')
print('-------------------------------')
print('Data types of the features in article master')
print(df_master.dtypes)
print('-------------------------------')
print('-------------------------------')

Size of the Raw Data  (964712, 22)
-------------------------------
-------------------------------
Data types of the features in article master
Circle                           object
City                             object
Partition_date                   object
Component                       float64
Hour                              int32
Sum_total_volume_mb_download     object
Sum_total_volume_mb_upload       object
Unique_customers                  int64
Total_duration_sec                int64
Total_sessions                    int64
Rsrp_nom                         object
Rsrp_dnom                         int64
Rsrq_nom                         object
Rsrq_dnom                         int64
Mean_cqi_nom                    float64
Mean_cqi_dnom                     int64
Pusch_sinr_nom                   object
Pusch_sinr_dnom                   int64
Ta_distance_nom                  object
Ta_distance_dnom                  int64
Intersite_distance_nom           object
Intersite_distan

In [9]:
### Basic QC's Check
print('No of Unique Circle ', df_master['Circle'].unique())
print('-----------------------------')
print('No of Cities in the circle',df_master[['Circle','City']].groupby('Circle').nunique())
print('-----------------------------')
print('No of Component in the circle',df_master[['Circle','Component']].groupby('Circle').nunique())
print('-----------------------------')
# print('Unique Anomaly',df_master['Anomaly'].unique())
# print('-----------------------------')
# print('Unique Cluster Issue',df_master['Cluster_issue'].unique())

No of Unique Circle  ['bihar']
-----------------------------
No of Cities in the circle         City
Circle      
bihar     56
-----------------------------
No of Component in the circle         Component
Circle           
bihar         264
-----------------------------


In [11]:
df_master.dropna(subset=['Component','City'],axis=0,inplace=True)
df_master['Component']=df_master['Component'].astype('int')

In [12]:
df_master.isnull().sum()

Circle                             0
City                               0
Partition_date                     0
Component                          0
Hour                               0
Sum_total_volume_mb_download       0
Sum_total_volume_mb_upload         0
Unique_customers                   0
Total_duration_sec                 0
Total_sessions                     0
Rsrp_nom                           0
Rsrp_dnom                          0
Rsrq_nom                           0
Rsrq_dnom                          0
Mean_cqi_nom                       0
Mean_cqi_dnom                      0
Pusch_sinr_nom                  7574
Pusch_sinr_dnom                    0
Ta_distance_nom                 1449
Ta_distance_dnom                   0
Intersite_distance_nom          1449
Intersite_distance_dnom            0
dtype: int64

In [13]:
df_master.head()

,Circle,City,Partition_date,Component,Hour,Sum_total_volume_mb_download,Sum_total_volume_mb_upload,Unique_customers,Total_duration_sec,Total_sessions,...,Rsrq_nom,Rsrq_dnom,Mean_cqi_nom,Mean_cqi_dnom,Pusch_sinr_nom,Pusch_sinr_dnom,Ta_distance_nom,Ta_distance_dnom,Intersite_distance_nom,Intersite_distance_dnom
0,bihar,hajipur,2022-11-04,98,17,44547835768.00,4727227696.00,4672,840864,49706,...,-562649.00,39294,392691.0,49276,146593.70,12682,56784132.00,49706,83771883.00,49706
1,bihar,jamui,2022-11-04,56,17,45228882630.00,4137939740.00,5133,1598569,43598,...,-398315.50,28126,376384.0,43409,326344.50,22993,13896817.00,43598,12780028.00,43598
2,bihar,daltonganj,2022-11-04,9,17,42583194734.00,3876797554.00,4902,1257292,43465,...,-461172.50,34509,389276.0,43192,297332.90,19718,10627596.00,43465,11452453.00,43465
3,bihar,sitamarhi,2022-11-04,27,17,75229462619.00,6707864085.00,8134,2627563,78809,...,-830623.50,55742,622129.0,78306,493853.90,42966,42858573.00,78809,33896908.00,78809
4,bihar,gaya,2022-11-04,112,17,110061710390.00,10600855782.00,8041,3021133,89193,...,-799238.00,60384,806191.0,88822,635052.00,45805,20777164.00,89193,26351571.00,89193


In [17]:
### Saving in Pre Processing Folder
df_master.to_parquet(pickle_path+'{}{}_Lsr_data.parquet'.format(monthname,year),index=False)
print('Data succesfully stored in:',pickle_path+'{}{}_Lsr_data.csv'.format(monthname,year))

Data succesfully stored in: /Users/raj2.gaurav/Desktop/Causal Model/02. Processed Data/03. LSR Data/Nov2022_Lsr_data.csv


In [16]:
path_dir:str =  "/Users/raj2.gaurav/Desktop/Causal Model/03. Collated Data/03. LSR Data/"
os.chdir(path_dir)
print("current dir is: %s" % (os.getcwd()))
file='Collated_Anomaly_Data.csv'

current dir is: /Users/raj2.gaurav/Desktop/Causal Model/03. Collated Data/03. LSR Data


In [ ]:
if os.path.isfile(file):
    print("File exists")
    pickle_path_col = loc_path + 'Causal Model/03. Collated Data/01. Anomaly Data/Collated_Anomaly_Data.csv'
    print('Reading master Anomaly data for collation......')
    collated_df = pd.read_csv(pickle_path)
    print('Data read, Shape of Collated data before collation:',collated_df.shape)
    collated_df = pd.concat([collated_df, df_master])
    collated_df = collated_df.drop_duplicates()
    collated_df.reset_index(drop = True, inplace = True)
    print('Data collated, Shape of Anomaly data after collation:',collated_df.shape)
    collated_df.to_csv(pickle_path,index=False)
else:
    print("File Doesn't exist")
        #os.mkdir(folder)
    print('Creating a master Anomaly data file......')
    collated_df = pd.DataFrame()
    path = loc_path + r'/Causal Model/02. Processed Data/01. Anomaly Data/'
    pickle_path = loc_path + 'Causal Model/03. Collated Data/01. Anomaly Data/'
    collated_file_name = []
    for subdir, dirs, files in os.walk(path):
        mis_df_collated_loop = pd.DataFrame()
        for file in files:
            try:
                file_path = os.path.join(path, subdir + '/' +file)
                suffix = subdir[-9:].lower()
                collated_file_name.append(file_path)
            except:
                pass

    for file in collated_file_name:
        print(str(file))
        collated_df = pd.concat([collated_df, pd.read_csv(file)])
        print('Master Anomaly data file created, Shape of Data:',collated_df.shape)
    print(pickle_path+'Collated_Anomaly_Data.csv')
    collated_df = collated_df.drop_duplicates()
    collated_df.to_csv(pickle_path+'Collated_Anomaly_Data.csv',index=False)